# Quiz 11

In [1]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import scipy

In [ ]:
detections = 0
for simulation in range(1000):
    num = 1000 
    
    event_time = int(num / 2) 
    
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
    
    R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num / 2) + 1) * 2 
    
    results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit() 
    
    alpha, beta = results.params 
    
    resid = R_target - results.predict(sm.add_constant(R_market)) 

    # See if t-statistic for event dummy is significant
    # Create event dummy
    # Note: event dummy is 1 at event_time + 1, else 0
    # We use event_time + 1 to account for the fact that the event effect is modeled to occur right after the event time
    # This is consistent with how we generated R_target above
    event_dummy = np.zeros(num)
    event_dummy[event_time + 1] = 1
    X_event = sm.add_constant(np.column_stack((R_market, event_dummy)))
    model_event = sm.OLS(R_target, X_event).fit()
    t_stat_event = model_event.tvalues[2]  # t-statistic for event dummy

    if abs(t_stat_event) > scipy.stats.t.ppf(0.975, df=num - 3):
        detections += 1
print(f"Detection Rate: {detections / 1000:.2%}")


Detection Rate: 49.70%


In [14]:
detections = 0
np.random.seed(0)
for simulation in range(1000):
    num = 1000 
    
    event_time = simulation % num
    
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
    
    R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num / 2) + 1) * 2 
        
    R_target[event_time] += 1  # event shock

    event_dummy = np.zeros(num)
    event_dummy[event_time] = 1

    X = sm.add_constant(np.column_stack((R_market, event_dummy)))
    model = sm.OLS(R_target, X).fit()
    t_stat = model.tvalues[2]

    cutoff = scipy.stats.t.ppf(0.975, df=num - 3)

    if abs(t_stat) > cutoff:
        detections += 1

rate = detections / 1000
print("Detection Rate:", rate)


Detection Rate: 0.175


In [16]:
detections = 0
np.random.seed(0)
for time in np.arange(480, 521):
    for simulation in range(1000):
        num = 1000 
        
        event_time = time
        
        R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
        
        R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num / 2) + 1) * 2 
            
        R_target[event_time] += 1  # event shock

        event_dummy = np.zeros(num)
        event_dummy[event_time] = 1

        X = sm.add_constant(np.column_stack((R_market, event_dummy)))
        model = sm.OLS(R_target, X).fit()
        t_stat = model.tvalues[2]

        cutoff = scipy.stats.t.ppf(0.975, df=num - 3)

        if abs(t_stat) > cutoff:
            detections += 1

rate = detections / 1000
print("Detection Rate:", rate)


Detection Rate: 7.7


In [17]:
def make_error(corr_const, num): 
    sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2)) 
    err = list() 
    prev = np.random.normal(0, sigma) 
    for n in range(num): 
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma) 
        err.append(prev) 
    return np.array(err)

detections = 0
np.random.seed(0)
for simulation in range(1000):
    num = 1000 
    
    event_time = simulation % num
    
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
    
    R_target = 2 + R_market + make_error(corr_const=0.9, num=num) + (np.arange(num) == int(num / 2) + 1) * 2 
        
    R_target[event_time] += 1  # event shock

    event_dummy = np.zeros(num)
    event_dummy[event_time] = 1

    X = sm.add_constant(np.column_stack((R_market, event_dummy)))
    model = sm.OLS(R_target, X).fit()
    t_stat = model.tvalues[2]

    cutoff = scipy.stats.t.ppf(0.975, df=num - 3)

    if abs(t_stat) > cutoff:
        detections += 1

rate = detections / 1000
print("Detection Rate:", rate)


Detection Rate: 0.059
